# 05 angr symbolic memory

Let's analyze this assembly code:
```
                             **************************************************************
                             *                          FUNCTION                          *
                             **************************************************************
                             int __cdecl main(int argc, char * * argv)
        00100854 ff c3 01 d1     sub        sp,sp,#0x70
        00100858 fd 7b 06 a9     stp        x29,x30,[sp, #local_10]
        0010085c fd 83 01 91     add        x29,sp,#0x60
        00100860 08 00 00 d0     adrp       x8,0x102000
        00100864 08 61 32 91     add        x8,x8,#0xc98
        00100868 e8 07 00 f9     str        x8=>user_input,[sp, #local_68]                   = ??
        0010086c e9 03 08 aa     mov        x9,x8
        00100870 29 21 00 91     add        x9,x9,#0x8
        00100874 e9 0b 00 f9     str        x9=>CHAR_ARRAY_00102c98+8,[sp, #local_60]        = ??
        00100878 e9 03 08 aa     mov        x9,x8
        0010087c 29 41 00 91     add        x9,x9,#0x10
        00100880 e9 0f 00 f9     str        x9=>CHAR_ARRAY_00102c98+16,[sp, #local_58]       = ??
        00100884 e9 03 08 aa     mov        x9,x8
        00100888 29 61 00 91     add        x9,x9,#0x18
        0010088c e9 13 00 f9     str        x9=>CHAR_ARRAY_00102c98+24,[sp, #local_50]       = ??
        00100890 bf c3 1d b8     stur       wzr,[x29, #local_34]
        00100894 a0 83 1d b8     stur       argc,[x29, #local_38]
        00100898 e1 1b 00 f9     str        argv,[sp, #local_40]
        0010089c a8 83 1f f8     stur       x8=>user_input,[x29, #local_18]                  = ??
        001008a0 28 04 80 d2     mov        x8,#0x21
        001008a4 a8 03 1f f8     stur       x8,[x29, #local_20]
        001008a8 bf c3 1e b8     stur       wzr,[x29, #local_24]
        001008ac a8 03 1e f8     stur       x8,[x29, #local_30]
        001008b0 a0 83 5f f8     ldur       argc=>user_input,[x29, #local_18]                = ??
        001008b4 a1 c3 5e b8     ldur       argv,[x29, #local_24]
        001008b8 a2 03 5e f8     ldur       x2,[x29, #local_30]
        001008bc a3 03 5f f8     ldur       x3,[x29, #local_20]
        001008c0 48 00 00 94     bl         <EXTERNAL>::__memset_chk                         undefined __memset_chk()
        001008c4 00 00 00 90     adrp       argc,0x100000
        001008c8 00 20 18 91     add        argc=>s_Enter_the_password:_00100608,argc,#0x608 = "Enter the password: "
        001008cc 3d 00 00 94     bl         <EXTERNAL>::printf                               int printf(char * __format, ...)
        001008d0 e1 07 40 f9     ldr        argv=>user_input,[sp, #local_68]                 = ??
        001008d4 e2 0b 40 f9     ldr        x2=>CHAR_ARRAY_00102c98+8,[sp, #local_60]        = ??
        001008d8 e3 0f 40 f9     ldr        x3=>CHAR_ARRAY_00102c98+16,[sp, #local_58]       = ??
        001008dc e4 13 40 f9     ldr        x4=>CHAR_ARRAY_00102c98+24,[sp, #local_50]       = ??
        001008e0 00 00 00 90     adrp       argc,0x100000
        001008e4 00 54 19 91     add        argc=>s_%8s_%8s_%8s_%8s_00100655,argc,#0x655     = "%8s %8s %8s %8s"
        001008e8 42 00 00 94     bl         <EXTERNAL>::scanf                                int scanf(char * __format, ...)
        001008ec ff 2f 00 b9     str        wzr,[sp, #local_44]

```

First we load the compiled binary executable:

In [20]:
import angr, claripy, sys

libname = "../build/arm64-v8a/05_angr_symbolic_memory"
project = angr.Project(libname, main_opts={ "base_addr": 0x00100000 })
print(project.arch, hex(project.entry))

<Arch AARCH64 (LE)> 0x10074c


We want start our execetion after invoke `scanf` function:(e.g.: 0x001008ec)

In [21]:
after_scanf_address = 0x001008ec # int(input("after scanf address(hex): "), 16)

We prepare the **blank state** for symbolic execution:

In [22]:
blank_state = project.factory.blank_state(addr=after_scanf_address)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_MEMORY)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_REGISTERS)

Let's create some symbolic memory which pass as param into `scanf` function:
```
        001008d0 e1 07 40 f9     ldr        argv=>user_input,[sp, #local_68]                 = ??
        001008d4 e2 0b 40 f9     ldr        x2=>CHAR_ARRAY_00102c98+8,[sp, #local_60]        = ??
        001008d8 e3 0f 40 f9     ldr        x3=>CHAR_ARRAY_00102c98+16,[sp, #local_58]       = ??
        001008dc e4 13 40 f9     ldr        x4=>CHAR_ARRAY_00102c98+24,[sp, #local_50]       = ??
        001008e0 00 00 00 90     adrp       argc,0x100000
```

`user_input` is a global data which inside `.bss` segment:
```
                             user_input                                                   main:001008d0(*), 
        00102c98                 char[33]   ??
           00102c98 [0]             ??,  ??,  ??,  ??
           00102c9c [4]             ??,  ??,  ??,  ??
           00102ca0 [8]             ??,  ??,  ??,  ??
           00102ca4 [12]            ??,  ??,  ??,  ??
           00102ca8 [16]            ??,  ??,  ??,  ??
           00102cac [20]            ??,  ??,  ??,  ??
           00102cb0 [24]            ??,  ??,  ??,  ??
           00102cb4 [28]            ??,  ??,  ??,  ??
           00102cb8 [32]            ??
```


In [23]:
x1_mem = claripy.BVS('x1', 8 * 8)
x2_mem = claripy.BVS('x2', 8 * 8)
x3_mem = claripy.BVS('x3', 8 * 8)
x4_mem = claripy.BVS('x4', 8 * 8)

blank_state.memory.store(0x00102c98, x1_mem, size=8)
blank_state.memory.store(0x00102c98 + 8, x2_mem, size=8)
blank_state.memory.store(0x00102c98 + 16, x3_mem, size=8)
blank_state.memory.store(0x00102c98 + 24, x4_mem, size=8)

Everything is ready, next we start the symbolic execution to find the path:

In [24]:
def is_successful(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Good Job.".encode() in stdout_output 

def should_abort(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Try again.".encode() in stdout_output

simulation = project.factory.simulation_manager(blank_state)
simulation.explore(find=is_successful, avoid=should_abort)
if simulation.found:
    solution1 = simulation.found[0].solver.eval(x1_mem, cast_to=bytes).decode()
    solution2 = simulation.found[0].solver.eval(x2_mem, cast_to=bytes).decode()
    solution3 = simulation.found[0].solver.eval(x3_mem, cast_to=bytes).decode()
    solution4 = simulation.found[0].solver.eval(x4_mem, cast_to=bytes).decode()

    solution = ' '.join([ solution1, solution2, solution3, solution4 ]) 
    print("Solution(password):", solution)
else:
    print("Cound not find the solution")
    list(map(lambda s: s.callstack.dbg_repr(), simulation.deadended))

Solution(password): PGJUTUKY GYOVDMUL TYKHGVMT XYJPGUTV
